In [46]:
import pandas as pd
from datetime import timedelta
from matplotlib import pyplot as plt
import numpy as np

Start by making DF for average temperature per day

In [47]:
df = pd.read_csv('../data/smhi_temp_data_save.csv', usecols=[0, 1, 2, 3], header=6, sep=';')

In [48]:
df

,Datum,Tid (UTC),Lufttemperatur,Kvalitet
0,1944-01-01,07:00:00,1.6,G
1,1944-01-01,13:00:00,3.8,G
2,1944-01-01,18:00:00,4.2,G
3,1944-01-02,07:00:00,0.6,G
4,1944-01-02,13:00:00,3.8,G
...,...,...,...,...
435387,2006-12-05,09:00:00,9.4,G
435388,2006-12-05,10:00:00,10.1,Y
435389,2006-12-05,11:00:00,10.8,Y
435390,2006-12-05,12:00:00,10.9,G


In [49]:
df['Snittemperatur'] = df.groupby('Datum')['Lufttemperatur'].transform('mean').round(2)

In [50]:
df = df.groupby('Datum').max().drop('Lufttemperatur', axis=1).reset_index()

In [51]:
df['Year'] = df['Datum'].str.split('-').str[0]

In [52]:
df['Year'] = pd.to_datetime(df['Year'], yearfirst=True, format='%Y')

In [53]:
df['Datum'] = pd.to_datetime(df['Datum'], yearfirst=True, format='ISO8601')

In [54]:
df

,Datum,Tid (UTC),Kvalitet,Snittemperatur,Year
0,1944-01-01,18:00:00,G,3.20,1944-01-01
1,1944-01-02,18:00:00,G,3.60,1944-01-01
2,1944-01-03,18:00:00,G,2.80,1944-01-01
3,1944-01-04,18:00:00,G,-4.47,1944-01-01
4,1944-01-05,18:00:00,G,-1.40,1944-01-01
...,...,...,...,...,...
22980,2006-12-01,23:00:00,Y,8.03,2006-01-01
22981,2006-12-02,23:00:00,Y,8.81,2006-01-01
22982,2006-12-03,23:00:00,Y,8.11,2006-01-01
22983,2006-12-04,23:00:00,Y,8.57,2006-01-01


In [55]:
df.dtypes

Datum             datetime64[ns]
Tid (UTC)                 object
Kvalitet                  object
Snittemperatur           float64
Year              datetime64[ns]
dtype: object

In [56]:
df.to_csv('../data/Olof_Save_avg_daily_temp_data.csv', index=False, mode='w')

Average temperature per month

In [61]:
# dtype_map = {'Datum': 'datetime64[ns]', 'Tid': object, 'Kvalitet': object, 'Snittemperatur': float, 'Year': 'datetime64[ns]'}

In [63]:
df_month = pd.read_csv('../data/Olof_Save_avg_daily_temp_data.csv', parse_dates=['Datum', 'Year'])

In [64]:
df_month.dtypes

Datum             datetime64[ns]
Tid (UTC)                 object
Kvalitet                  object
Snittemperatur           float64
Year              datetime64[ns]
dtype: object